# Get Data Ready

In [1]:
#!pip install gdown

In [1]:
#https://drive.google.com/file/d/1OTwtZadt0VpdbK3oo7ap3FIANzzdBcx6/view?usp=sharing, 
!gdown --id 1OTwtZadt0VpdbK3oo7ap3FIANzzdBcx6
#https://drive.google.com/file/d/1Wbc1RDWpw74GaX7W7l8Kl2ymNqoj87uV/view?usp=sharing
!gdown --id 1Wbc1RDWpw74GaX7W7l8Kl2ymNqoj87uV

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1OTwtZadt0VpdbK3oo7ap3FIANzzdBcx6
To: /home/ec2-user/SageMaker/danmu_text_6_cates.pkl
100%|███████████████████████████████████████| 3.39M/3.39M [00:00<00:00, 123MB/s]
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Wbc1RDWpw74GaX7W7l8Kl2ymNqoj87uV
To: /home/ec2-user/SageMaker/danmu_label_6_cates.pkl
100%|████████████████████████████████████████| 300k/300k [00:00<00:00, 35.3MB/s]


In [4]:
import pickle

token_file = open("danmu_text_6_cates.pkl", "rb")
dist_file = open("danmu_label_6_cates.pkl", "rb")

tokens = pickle.load(token_file)
dists = pickle.load(dist_file)

print(len(tokens))
print(len(dists))

149981
149981


In [5]:
import numpy as np
labels = dists

label_distribution = {}
for i in labels:
    if i in label_distribution:
        label_distribution[i] += 1
    else:
        label_distribution[i] = 1

In [6]:
len(label_distribution)

6

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.DataFrame({
    'text': tokens,
    'label': labels
})

train_df, test_df = train_test_split(df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.1)

In [8]:
#Debugging
train_df = train_df[train_df['text'] != 'nan']
test_df = test_df[test_df['text'] != 'nan']
val_df = val_df[val_df['text'] != 'nan']


In [9]:
train_df.reset_index(drop=True)
test_df.reset_index(drop=True)
val_df.reset_index(drop=True)

,text,label
0,居然提前,3
1,糍糕特别好吃,4
2,侏罗纪,5
3,指甲剪,3
4,点赞数,1
...,...,...
11994,天下第一,5
11995,冯克雷真,0
11996,华容道啊,0
11997,这个我也去过,4


In [10]:
#save to csv

train_df = train_df.dropna()
test_df = test_df.dropna()
val_df = val_df.dropna()

train_df.to_csv('data_csv/train.csv', index = False)
test_df.to_csv('data_csv/test.csv', index = False)
val_df.to_csv('data_csv/val.csv', index = False)

# train_dict = train_df.to_dict()
# test_dict = test_df.to_dict()
# val_dict = val_df.to_dict()
# train_data_json = {
#     'train' : train_dict,
#     'test' : test_dict,
#     'validation': val_dict
# }

# import json

# with open('data_csv/data.json', 'w') as f:
#     json.dump(train_data_json, f)
    

# Preprocessing

In [11]:
import datasets
train = datasets.Dataset.from_pandas(train_df, preserve_index=False)
val = datasets.Dataset.from_pandas(val_df, preserve_index=False)
test = datasets.Dataset.from_pandas(test_df, preserve_index=False)

In [12]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

In [13]:
#Tokenization

tokenized_train =  train.map(preprocess_function, batched=True)
tokenized_val =  val.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

  0%|          | 0/108 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/30 [00:00<?, ?ba/s]

In [14]:
# For batching and padding
import transformers 

data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
#!conda install pytorch torchvision torchaudio cpuonly -c pytorch

# Model

In [15]:
# model = transformers.AutoModelForSequenceClassification.from_pretrained("uer/roberta-base-finetuned-jd-full-chinese",\
#                                                                         num_labels = len(label_distribution), \
#                                                                        ignore_mismatched_sizes=True)
model = transformers.AutoModelForSequenceClassification.from_pretrained("bert-base-chinese", num_labels = len(label_distribution))

training_args = transformers.TrainingArguments(
    output_dir="chinese_base",
    
)

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [16]:
#metrics

import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [17]:
trainer = transformers.Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_train,
    eval_dataset = tokenized_val,
    data_collator=data_collator,
    tokenizer = tokenizer,
)

In [19]:
#trainer.train()

# SageMaker

In [18]:
import sagemaker

session = sagemaker.Session()

session_bucket = None
if session_bucket == None and session is not None:
    session_bucket = session.default_bucket()
    
role = sagemaker.get_execution_role()
session = sagemaker.Session(default_bucket = session_bucket)

print(f'role: {role}')
print(f'bucket: {session_bucket}')
print(f'session region: {session.boto_region_name}')

role: arn:aws:iam::635837196364:role/service-role/AmazonSageMaker-ExecutionRole-20220511T210233
bucket: sagemaker-us-east-1-635837196364
session region: us-east-1


Upload to S3 bucket

In [19]:
import botocore
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem()
s3_prefix = 'dataset/danmu_main'

#save data to S3
train_input_path = f's3://{session.default_bucket()}/{s3_prefix}/train'
#train_dataset.save_to_disk(train_input_path, fs = s3)
session.upload_data(path='data_csv/train.csv', bucket=session_bucket, key_prefix=s3_prefix+'/train')

test_input_path = f's3://{session.default_bucket()}/{s3_prefix}/test'
#test_dataset.save_to_disk(test_input_path, fs = s3)
session.upload_data(path='data_csv/test.csv', bucket=session_bucket, key_prefix=s3_prefix+'/test')

val_input_path = f's3://{session.default_bucket()}/{s3_prefix}/val'
session.upload_data(path='data_csv/val.csv', bucket=session_bucket, key_prefix=s3_prefix+'/val')


's3://sagemaker-us-east-1-635837196364/dataset/danmu_main/val/val.csv'

In [20]:
import sagemaker
from sagemaker.huggingface import HuggingFace

# gets role for executing training job

hyperparameters = {
    'model_name_or_path':'bert-base-chinese',
    'output_dir':'/opt/ml/model/bert_base_chinese',
    # add your remaining hyperparameters 
    # more info here https://github.com/huggingface/transformers/tree/v4.17.0/examples/pytorch/text-classification
    'max_seq_length':128,
    'per_device_train_batch_size' : 24,
    'learning_rate' : 2e-5,
    'num_train_epochs': 5,
    #train
    'do_train': True,
    #'do_eval' : True,
    #data
    'train_file': '/opt/ml/input/data/train/train.csv',
    #'test_file': '/opt/ml/input/data/train/test.csv',
    'validation_file': '/opt/ml/input/data/val/val.csv',
}

In [21]:
# git configuration to download our fine-tuning script
git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.17.0'}

# creates Hugging Face estimator
huggingface_estimator = HuggingFace(
    #entry_point='run_glue_t5.py',
    entry_point='run_glue.py',
    #source_dir='./examples/pytorch/text-classification',
    source_dir = './scripts',
    instance_type='ml.g5.xlarge',
    instance_count=1,
    role=role,
    #it_config=git_config,
    transformers_version='4.17.0',
    pytorch_version='1.10.2',
    py_version='py38',
    hyperparameters = hyperparameters
)


In [ ]:

# starting the train job
huggingface_estimator.fit({'train': train_input_path, 'test': test_input_path, 'val':val_input_path})

2022-05-12 15:23:27 Starting - Starting the training job...
2022-05-12 15:23:51 Starting - Preparing the instances for trainingProfilerReport-1652369006: InProgress
.........
2022-05-12 15:25:19 Downloading - Downloading input data...
2022-05-12 15:25:39 Training - Downloading the training image..........................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-05-12 15:30:12,662 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-05-12 15:30:12,683 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-05-12 15:30:12,691 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-05-12 15:30:13,292 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "test": "/opt/ml/input/data/test",
        "train": "/opt/ml/

In [9]:
#attach back to trainning job if needed:
import sagemaker
import boto3

need_attach = False
session = sagemaker.Session()
if need_attach:
    output = sagemaker.estimator.Estimator.attach('huggingface-pytorch-training-2022-05-12-02-25-20-722', sagemaker_session = session)
    print(output)

# Eval

In [ ]:
import transformers
from sagemaker.huggingface import HuggingFace
from sagemaker.huggingface.model import HuggingFaceModel

import_from_s3 = False

if import_from_s3:
    #To restore old trainning job
    # create Hugging Face Model Class
    huggingface_model = HuggingFaceModel(
        model_data="s3://sagemaker-us-east-1-635837196364/huggingface-pytorch-training-2022-05-12-00-16-12-039/output/model.tar.gz",  # path to your trained SageMaker model
        role=role,                                            # IAM role with permissions to create an endpoint
        transformers_version='4.17.0',                        # Transformers version used
        pytorch_version='1.10.2',                             # PyTorch version used
        py_version='py38',                                # Python version used
        env = {
            'HF_MODEL_ID':'uer/chinese_roberta_L-12_H-768',
            'HF_TASK': 'text-classification',

        },
    )
    # deploy model to SageMaker Inference
    predictor = huggingface_model.deploy(
       initial_instance_count=1,
       instance_type="ml.m5.4xlarge"
    )
else:
    predictor = huggingface_estimator.deploy(1, "ml.m5.4xlarge")

In [ ]:
input = ["input": '你好呀']

predictor.predict(input)

In [ ]:
#predictor.delete_endpoint()